In [1]:
from keras.datasets import mnist

(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

Using TensorFlow backend.


In [2]:
import numpy as np
# train and test images in shape (60000, 28, 28)
train_images_28x28 = train_images_original.astype('float32') / 255
test_images_28x28 = test_images_original.astype('float32') / 255

train_images_rounded = np.round(train_images_28x28)
test_images_rounded = np.round(test_images_28x28)

# train and test images in shape (60000, 784)
train_images = train_images_original.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images_original.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

In [3]:
def neighbours(x, y, zeros):
    DIRC = [(0, 1), (1, 0), (0, -1), (-1, 0),
            (1, 1), (1, -1), (-1, 1), (-1, -1)]
    ROW = 28
    COL = 28

    nb = []
    for d in DIRC:
        dx, dy = d
        new_x = x + dx
        new_y = y + dy
        if 0 <= new_x < COL and 0 <= new_y < ROW:
            if [new_x, new_y] in zeros:
                new_point = [new_x, new_y]
                nb.append(new_point)
    return nb

0


KeyboardInterrupt: 

In [39]:
def islands(zeros):
    visited = []
    count = 0

    for p in zeros:
        p = [p[0], p[1]]
        if p not in visited:
            count += 1
            dfs(p, visited, zeros)

    return float(count)

0


KeyboardInterrupt: 

In [5]:
def dfs(point, visited, zeros):
    point = [point[0], point[1]]
    if point in visited:
        return
    else:
        visited.append(point)
        nb = neighbours(point[0], point[1], zeros)

        for n in nb:
            dfs(n, visited, zeros)

0


KeyboardInterrupt: 

In [50]:
train_images_featured = np.empty((0, 785))
for i in range(train_images.shape[0]):
    if i % 10000 == 0:
        print(i)
    allZeros = np.argwhere(train_images_rounded[i] == 0.0)
    allZeros = allZeros.tolist()
    islandsNum = islands(allZeros)
    im = np.append(train_images[i], islandsNum)
    train_images_featured = np.vstack([train_images_featured, im])

0


KeyboardInterrupt: 

In [42]:
test_images_featured = np.empty((0, 785))
for i in range(test_images.shape[0]):
    if i % 1000 == 0:
        print(i)
    allZeros = np.argwhere(test_images_rounded[i] == 0.0)
    allZeros = allZeros.tolist()
    islandsNum = islands(allZeros)
    im = np.append(test_images[i], islandsNum)
    test_images_featured = np.vstack([test_images_featured, im])


In [43]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(785,)))
network.add(layers.Dense(10, activation='softmax'))


In [44]:
network.compile(optimizer='sgd',
                loss='categorical_crossentropy',
                metrics=['accuracy'])


In [45]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels_original)
test_labels = to_categorical(test_labels_original)


In [51]:
epochs = 10
history = network.fit(train_images_featured, 
                      train_labels, 
                      epochs=epochs, 
                      batch_size=128, 
                      validation_data=(test_images_featured, test_labels))

ValueError: Input arrays should have the same number of samples as target arrays. Found 70 input samples and 60000 target samples.

In [ ]:
loss, accuracy = network.evaluate(test_images, test_labels, verbose=False)
print("Test loss: ", round(loss, 3))
print("Test accuracy: ", round(accuracy, 3))
